In [ ]:
import Model
import utils
import pickle
import numpy as np
import keras
from importlib import reload
from settings import in_size, out_size, train_start, train_end, valid_start, valid_end
from utils.Model import create_gnn_seq2seq
%matplotlib notebook

In [ ]:
Model.set_gpu("7")

In [ ]:
_ = pickle.load(open("./data/data.pickle","rb"))
cluster_sizes = _["cluster_sizes"]
cluster2id = _["cluster2id"]
locations = _["locations"]
timestamps = _["timestamps"]
raw_data = _["raw_data"]
cluster_data = _["cluster_data"]
onehot_data = _["onehot_data"]
half = 77042
one_year = 59570
one_year_and_half = 42098
two_years = 24626
full = 8500

In [ ]:
train_full = utils.data_iterator(cluster_data[train_start:train_end],timestamps[train_start:train_end],in_size,out_size=out_size,slide=1)
train_half = utils.data_iterator(cluster_data[half:train_end],timestamps[half:train_end],in_size,out_size=out_size,slide=1)
train_one = utils.data_iterator(cluster_data[one_year:train_end],timestamps[one_year:train_end],in_size,out_size=out_size,slide=1)
train_one_half = utils.data_iterator(cluster_data[one_year_and_half:train_end],timestamps[one_year_and_half:train_end],in_size,out_size=out_size,slide=1)
train_two = utils.data_iterator(cluster_data[two_years:train_end],timestamps[two_years:train_end],in_size,out_size=out_size,slide=1)

valid = utils.data_iterator(cluster_data[valid_start:valid_end],timestamps[valid_start:valid_end],in_size,out_size=out_size,slide=1)
train_onehot = utils.data_generator(onehot_data[train_start:train_end],timestamps[train_start:train_end],in_size,out_size=out_size,slide=1)
valid_onehot = utils.data_iterator(onehot_data[valid_start:valid_end],timestamps[valid_start:valid_end],in_size,out_size=out_size,slide=1)

In [ ]:
def callbacks(modelname):
    # callbacks を生成する関数
    
    # valid で最も良い性能になったモデルを保存する
    mc_cb = keras.callbacks.ModelCheckpoint(
        './models/'+modelname+'.clf', 
        monitor='val_loss', 
        verbose=1, 
        save_best_only=True,
        save_weights_only=False, 
        mode='auto', 
        period=1
    )

    # val_loss が7エポック以上減少しなかったら訓練終了
    es_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, verbose=1, mode='auto')
    
    # 2エポック以上 val_loss が減少しなかったら学習率を削減
    rl_cb = keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.3, 
        patience=2, 
        verbose=1, 
        mode='auto', 
        epsilon=0.00005, 
        cooldown=2, 
        min_lr=0
    )
    return [mc_cb,rl_cb]

In [ ]:
半年
s2s = Model.create_seq2seq(len(cluster2id),attr_size=4,pad_size=in_size,out_size=out_size,loss='mae')
history = s2s.fit(
    train_half[0],train_half[1],batch_size=512,epochs=100,
    validation_data=valid,shuffle=True,callbacks=callbacks('s2s_mae_half')
)

conv_s2s = Model.create_conv_encoder_seq2seq(len(cluster2id),attr_size=4,pad_size=in_size,out_size=out_size,loss='mae',lr=0.0001,latent_dim=128)
history = conv_s2s.fit(
    train_half[0],train_half[1],batch_size=512,epochs=100,
    validation_data=valid,shuffle=True,callbacks=callbacks('s2s_conv_mae_half')
)

create_gnn_seq2seq.fit("half", half)

In [ ]:
# 1年
s2s = Model.create_seq2seq(len(cluster2id),attr_size=4,pad_size=in_size,out_size=out_size,loss='mae')
history = s2s.fit(
    train_one[0],train_one[1],batch_size=512,epochs=100,
    validation_data=valid,shuffle=True,callbacks=callbacks('s2s_mae_one')
)

conv_s2s = Model.create_conv_encoder_seq2seq(len(cluster2id),attr_size=4,pad_size=in_size,out_size=out_size,loss='mae')
history = conv_s2s.fit(
    train_one[0],train_one[1],batch_size=512,epochs=100,
    validation_data=valid,shuffle=True,callbacks=callbacks('s2s_conv_mae_one')
)

create_gnn_seq2seq.fit("one", one_year)

In [ ]:
# 1年半
s2s = Model.create_seq2seq(len(cluster2id),attr_size=4,pad_size=in_size,out_size=out_size,loss='mae')
history = s2s.fit(
    train_one_half[0],train_one_half[1],batch_size=512,epochs=100,
    validation_data=valid,shuffle=True,callbacks=callbacks('s2s_mae_one_half')
)

conv_s2s = Model.create_conv_encoder_seq2seq(len(cluster2id),attr_size=4,pad_size=in_size,out_size=out_size,loss='mae')
history = conv_s2s.fit(
    train_one_half[0],train_one_half[1],batch_size=512,epochs=100,
    validation_data=valid,shuffle=True,callbacks=callbacks('s2s_conv_mae_one_half')
)

create_gnn_seq2seq.fit("one_half", one_year_and_half)

In [ ]:
# 2年
s2s = Model.create_seq2seq(len(cluster2id),attr_size=4,pad_size=in_size,out_size=out_size,loss='mae')
history = s2s.fit(
    train_two[0],train_two[1],batch_size=512,epochs=100,
    validation_data=valid,shuffle=True,callbacks=callbacks('s2s_mae_two')
)

conv_s2s = Model.create_conv_encoder_seq2seq(len(cluster2id),attr_size=4,pad_size=in_size,out_size=out_size,loss='mae')
history = conv_s2s.fit(
    train_two[0],train_two[1],batch_size=512,epochs=100,
    validation_data=valid,shuffle=True,callbacks=callbacks('s2s_conv_mae_two')
)

create_gnn_seq2seq.fit("two", two_years)

In [ ]:
# Full
s2s = Model.create_seq2seq(len(cluster2id),attr_size=4,pad_size=in_size,out_size=out_size,loss='mae')
history = s2s.fit(
    train_full[0],train_full[1],batch_size=512,epochs=100,
    validation_data=valid,shuffle=True,callbacks=callbacks('s2s_mae')
)

conv_s2s = Model.create_conv_encoder_seq2seq(len(cluster2id),attr_size=4,pad_size=in_size,out_size=out_size,loss='mae')
history = conv_s2s.fit(
    train_full[0],train_full[1],batch_size=512,epochs=100,
    validation_data=valid,shuffle=True,callbacks=callbacks('s2s_conv_mae')
)

create_gnn_seq2seq.fit("full", full)